# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,0.542096,1.0,296.0,15.3,1.000000,-1.275260
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,0.623954,2.0,242.0,17.8,1.000000,-0.263711
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,0.623954,2.0,242.0,17.8,0.989737,-1.627858
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,0.707895,3.0,222.0,18.7,0.994276,-2.153192
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,0.707895,3.0,222.0,18.7,1.000000,-1.162114


In [2]:
X = boston_features
y = pd.DataFrame(boston.target,columns = ['MEDV'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)

In [5]:
from sklearn.linear_model import LinearRegression

Fit the model and apply the model to the make test set predictions

In [6]:
linreg = LinearRegression()
linreg.fit(X_train,y_train)
y_hat_test = linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [8]:
from sklearn.metrics import mean_squared_error
resid_test = y_test - y_hat_test
MSE = mean_squared_error(y_test, y_hat_test)
print(MSE)

26.274691057677337


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [37]:
def kfolds(data, k):
    # Force data as pandas dataframe
    if not isinstance(data,pd.DataFrame):
        data_copy = data
        data = pd.DataFrame(data=data_copy)
    # add 1 to fold size to account for leftovers
    num_larger_datasets = len(data)%k
    length_datasets = int(len(data)/k)
    larger_bins = list(range(0,num_larger_datasets*length_datasets+length_datasets,length_datasets+1))
    smaller_bins = list(range(max(larger_bins)+length_datasets,len(data)+1,length_datasets))
    bins = larger_bins + smaller_bins

    if max(bins) != len(data):
        raise Exception('The length of the subsets is not equal to the length of the data')
    
    subsets = []
    for i in list(range(1,len(bins))):
        subsets.append(data.iloc[bins[i-1]:bins[i],:])
    
    return subsets

### Apply it to the Boston Housing Data

In [77]:
# Make sure to concatenate the data again
boston_df = pd.concat([X,y], axis=1)
subs = kfolds(boston_df,10)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [99]:
test = pd.concat(test_list,axis=0)
len(test)

404

In [101]:
test_errs = []
train_errs = []
k=5

subs = kfolds(boston_df,k)

for n in range(k):
    # Split in train and test for the fold
    train = subs[n]
    indeces = list(range(5))
    indeces.remove(n)
    test_list = [subs[i] for i in indeces]
    test = pd.concat(test_list,axis=0)
    # Fit a linear regression model
    linreg = LinearRegression()
    linreg.fit(train.drop(labels = 'MEDV', axis=1),train.MEDV)
    y_hat_test = linreg.predict(test.drop(labels = 'MEDV', axis=1))
    y_hat_train = linreg.predict(train.drop(labels = 'MEDV', axis=1))
    #Evaluate Train and Test Errors
    test_errs.append(mean_squared_error(test.MEDV, y_hat_test))
    train_errs.append(mean_squared_error(train.MEDV, y_hat_train))
    
print(train_errs)
print(test_errs)

[4.065271384236386, 7.679438322435297, 7.882048370007212, 17.909031530759645, 8.325592700521508]
[1150.0017015754356, 381.75122863880597, 1639.4310456005333, 40.98232470529148, 51.145996060452084]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [103]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")
cv_5_results

array([-13.04498602, -14.64079171, -24.85806428, -55.30023395,
       -19.21846011])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [105]:
print(np.mean(cv_5_results))
print(MSE)

-25.412507213763252
26.274691057677337


##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!